**420-A52-SF - Algorithmes d'apprentissage supervisé - Automne 2022 - Spécialisation technique en Intelligence Artificielle**<br/>
MIT License - Copyright (c) 2022 Mikaël Swawola

# Correction du projet #2 (classification)

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [6, 6]

## A - Chargement et préparation sommaire des données

#### Lecture de `chansons.csv`

In [ ]:
import pandas as pd
chansons = pd.read_csv('../../data/chansons.csv')

In [ ]:
chansons.head()

#### On ne garde pour le moment que les variables "acoustiques". Nous n'avons pas les outils nécéssaires pour gérer les variables catégorielles.

In [ ]:
X = chansons.drop(columns=['annee', 'titre', 'artiste', 'chansonID', 'artisteID', 'Top10'])
y = chansons['Top10']

#### Vérification de la proportion des classes positives (dans le Top10) et négatives (hors Top10) 

In [ ]:
ratio = y.sum()/len(y)
print(f'Ratio de la classe + : {ratio}')

Il y a un fort débalancement. En effet, ce ne sont pas tous les artistes qui ont la chance (ou le talent) de se retrouver dans le Top 10 !

#### Préparation du jeu de test

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)
# Cette fois-ci, nous appliqueront la standardiation après le train_test_split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, stratify=y, random_state=2023)

In [ ]:
print(f'X_train: {X_train.shape[0]} observations - Ratio de la classe + : {y_train.sum()/len(y_train)}')
print(f'X_test: {X_test.shape[0]} observations - Ratio de la classe + : {y_test.sum()/len(y_test)}')

#### On garde les indices d'origine au cas où...

In [ ]:
train_index = X_train.index
test_index = X_test.index

#### On applique la standardisation sur l'entraînement uniquement

In [ ]:
X_train_scaled = scaler.transform(X_train)

## B - Modélisation

### 1 - Régression logistique

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from helpers import plot_roc_curve

In [ ]:
# Grille de recherche
parameters = {'C':[0.01, 0.5, 1, 5, 10, 100],
              'l1_ratio':[0, 0.5, 0.9, 1]}

# Régression logistique
clf_logreg = LogisticRegression(penalty='elasticnet',
                                  max_iter=10000,
                                  solver='saga',
                                  n_jobs=-1,
                                  random_state=2023)

# GridSearch avec Validation croisée
clf_logreg_grid = GridSearchCV(clf_logreg, parameters, cv=5, scoring="neg_log_loss", verbose=1, n_jobs=-1, refit=True)

# Entraînement
clf_logreg_grid.fit(X_train_scaled, y_train)

In [ ]:
print(f'Meilleurs paramètres: {clf_logreg_grid.best_params_}')
print(f'Meilleur score (mean log loss CV): {-clf_logreg_grid.best_score_}')

In [ ]:
clf_logreg_final = clf_logreg_grid.best_estimator_
clf_logreg_final

#### Aire sous la courbe

In [ ]:
y_train_pred_proba_logreg = clf_logreg_final.predict_proba(X_train_scaled)[:,1]
print(f'AUC = {roc_auc_score(y_train, y_train_pred_proba_logreg)}')

| Modèle | Log loss | AUC (refit)
| ------ | ------ | ------
| Régression logisitique | 0.3345   | 0.8175

#### Courbe ROC

In [ ]:
results = {}
results['Logistic Regression'] = y_train_pred_proba_logreg
plot_roc_curve(results, y_train)

### 2 - K plus proches voisins

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# Grille de recherche
parameters = {
    "n_neighbors": [100, 110, 120, 130, 140, 150, 200],
    "weights": ["distance", "uniform"]
}

# KNN
clf_knn = KNeighborsClassifier(algorithm="brute")

# GridSearch avec Validation croisée
clf_knn_grid = GridSearchCV(clf_knn, parameters, cv=5, scoring="neg_log_loss", verbose=1, n_jobs=-1, refit=True)

# Entraînement
clf_knn_grid.fit(X_train_scaled, y_train)

In [ ]:
print(f'Meilleurs paramètres: {clf_knn_grid.best_params_}')
print(f'Meilleur score (mean log loss CV): {-clf_knn_grid.best_score_}')

In [ ]:
clf_knn_final = clf_knn_grid.best_estimator_
clf_knn_final

#### Aire sous la courbe

In [ ]:
y_train_pred_proba_knn = clf_knn_final.predict_proba(X_train_scaled)[:,1]
print(f'AUC = {roc_auc_score(y_train, y_train_pred_proba_knn)}')

| Modèle | Log loss | AUC (refit)
| ------ | ------ | ------
| Régression logisitique | 0.3345   | 0.8174  
| KNN | 0.3532 | 0.8166

#### Courbe ROC

In [ ]:
results['KNN'] = y_train_pred_proba_knn
plot_roc_curve(results, y_train)

### 3 - Arbres de décision

In [ ]:
from sklearn.utils.fixes import loguniform
from scipy.stats import randint
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# Distributions des paramètres
distributions = dict(
    criterion=['gini', 'entropy'],
    ccp_alpha=loguniform(1e-3, 1e3),
    max_depth=randint(2, 128))

# Arbre de décision
clf_tree = DecisionTreeClassifier(random_state=2023)

# Recherche aléatoire avec avec validation croisée
clf_tree_rnd = RandomizedSearchCV(clf_tree, distributions, n_iter=100, cv=5, scoring="neg_log_loss", verbose=2, n_jobs=-1, random_state=2023, refit=True)

# Entraînement
clf_tree_rnd.fit(X_train_scaled, y_train)

In [ ]:
print(f'Meilleurs paramètres: {clf_tree_rnd.best_params_}')
print(f'Meilleur score (mean log loss CV): {-clf_tree_rnd.best_score_}')

In [ ]:
clf_tree_final = clf_tree_rnd.best_estimator_
clf_tree_final

#### Aire sous la courbe

In [ ]:
y_train_pred_proba_tree = clf_tree_final.predict_proba(X_train_scaled)[:,1]
print(f'AUC = {roc_auc_score(y_train, y_train_pred_proba_tree)}')

| Modèle | Log loss | AUC (refit)
| ------ | ------ | ------
| Régression logisitique | 0.3345   | 0.8174  
| KNN | 0.3532 | 0.8166
| Arbres | 0.3780 | 0.7537

#### Courbe ROC

In [ ]:
results['Decision Tree'] = y_train_pred_proba_tree
plot_roc_curve(results, y_train)

### 4 - Bagging avec arbres

In [ ]:
from sklearn.ensemble import BaggingClassifier

In [ ]:
# Distributions des paramètres
distributions = dict(
    n_estimators=randint(2, 100))

# Bagging
clf_bag = BaggingClassifier(estimator=clf_tree_final, random_state=2023)

# Recherche aléatoire avec validation croisée
clf_bag_rnd = RandomizedSearchCV(clf_bag, distributions, n_iter=20, cv=5, scoring="neg_log_loss", verbose=1, n_jobs=-1, random_state=2023, refit=True)

# Entraînement
clf_bag_rnd.fit(X_train_scaled, y_train)

In [ ]:
print(f'Meilleurs paramètres: {clf_bag_rnd.best_params_}')
print(f'Meilleur score (mean log loss CV): {-clf_bag_rnd.best_score_}')

In [ ]:
clf_bag_final = clf_bag_rnd.best_estimator_
clf_bag_final

#### Aire sous la courbe

In [ ]:
y_train_pred_proba_bag = clf_bag_final.predict_proba(X_train_scaled)[:,1]
print(f'AUC = {roc_auc_score(y_train, y_train_pred_proba_bag)}')

| Modèle | Log loss | AUC (refit)
| ------ | ------ | ------
| Régression logisitique | 0.3345   | 0.8174  
| KNN | 0.3532 | 0.8166
| Arbres | 0.3780 | 0.7573
| Bagging (arbres) | 0.3452 | 0.8423

#### Courbe ROC

In [ ]:
results['Bagging (Tree)'] = y_train_pred_proba_bag
plot_roc_curve(results, y_train)

### 5 - Gradient boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
# Distributions des paramètres
distributions = dict(
    n_estimators=randint(2, 100),
    learning_rate=loguniform(1e-5, 1),
    max_depth=randint(2, 100),
    max_features=['sqrt', 'log2', None],
    loss=['log_loss', 'exponential'],
    ccp_alpha=loguniform(1e-6, 10)
)

# Gradient boosting
clf_gb = GradientBoostingClassifier(random_state=2023)


# Recherche aléatoire avec validation croisée
clf_gb_rnd = RandomizedSearchCV(clf_gb, distributions, n_iter=20, cv=5, scoring="neg_log_loss", verbose=1, n_jobs=-1, random_state=2023, refit=True)

# Entraînement
clf_gb_rnd.fit(X_train_scaled, y_train)

In [ ]:
clf_gb_final = clf_gb_rnd.best_estimator_
clf_gb_final

In [ ]:
print(f'Meilleurs paramètres: {clf_gb_rnd.best_params_}')
print(f'Meilleur score (mean log loss CV): {-clf_gb_rnd.best_score_}')

#### Aire sous la courbe

In [ ]:
y_train_pred_proba_gb = clf_gb_final.predict_proba(X_train_scaled)[:,1]
print(f'AUC = {roc_auc_score(y_train, y_train_pred_proba_gb)}')

| Modèle | Log loss | AUC (refit)
| ------ | ------ | ------
| Régression logisitique | 0.3345   | 0.8174  
| KNN | 0.3532 | 0.8166
| Arbres | 0.3780 | 0.7573
| Bagging (arbres) | 0.3452 | 0.8423
| Gradient boosting | 0.3452 | 0.8184

#### Courbe ROC

In [ ]:
results['Gradient Boosting'] = y_train_pred_proba_gb
plot_roc_curve(results, y_train)

### 6 - Forêts aléatoires

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Distributions des paramètres
distributions = dict(
    n_estimators=randint(2, 100),
    max_depth=randint(2, 100),
    max_features=['sqrt', 'log2', None],
    ccp_alpha=loguniform(1e-6, 10)
)

# Forêts aléatoires
clf_rf = RandomForestClassifier(random_state=2023)


# Recherche aléatoire avec validation croisée
clf_rf_rnd = RandomizedSearchCV(clf_rf, distributions, n_iter=100, cv=5, scoring="neg_log_loss", verbose=1, n_jobs=-1, random_state=2023, refit=True)

# Entraînement
clf_rf_rnd.fit(X_train_scaled, y_train)

In [ ]:
clf_rf_final = clf_rf_rnd.best_estimator_
clf_rf_final

In [ ]:
print(f'Meilleurs paramètres: {clf_rf_rnd.best_params_}')
print(f'Meilleur score (mean log loss CV): {-clf_rf_rnd.best_score_}')

#### Aire sous la courbe

In [ ]:
y_train_pred_proba_rf = clf_rf_final.predict_proba(X_train_scaled)[:,1]
print(f'AUC = {roc_auc_score(y_train, y_train_pred_proba_rf)}')

| Modèle | Log loss | AUC (refit)
| ------ | ------ | ------
| Régression logisitique | 0.3345   | 0.8174  
| KNN | 0.3532 | 0.8166
| Arbres | 0.3780 | 0.7573
| Bagging (arbres) | 0.3452 | 0.8423
| Gradient boosting | 0.3452 | 0.8221
| Forêts aléatoires | 0.3451 | 0.9796

#### Courbe ROC

In [ ]:
results['Random Forests'] = y_train_pred_proba_rf
plot_roc_curve(results, y_train)

### 7 - Ajout de l'ACP

#### On récupère les colonnes discrètes

In [ ]:
X_prime = chansons[['annee', 'titre', 'artiste']]
X_prime_ind = pd.get_dummies(X_prime, columns=['titre', 'artiste'], drop_first=True)

In [ ]:
X_prime_ind.shape

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

# ACP
pca = PCA(n_components=10)

# Entraînement
pca.fit(X_prime_ind)

In [ ]:
print(pca.explained_variance_ratio_)

In [ ]:
X_prime_all = np.c_[
    X_train_scaled,
    pca.transform(X_prime_ind)[train_index,:]
]

In [ ]:
X_prime_all.shape

In [ ]:
# Distributions des paramètres
distributions = dict(
    n_estimators=randint(2, 100),
    max_depth=randint(2, 100),
    max_features=['sqrt', 'log2', None],
    ccp_alpha=loguniform(1e-6, 10)
)

# Forêts aléatoires
clf_rf_acp = RandomForestClassifier(random_state=2023)


# Recherche aléatoire avec validation croisée
clf_rf_acp_rnd = RandomizedSearchCV(clf_rf_acp, distributions, n_iter=100, cv=5, scoring="neg_log_loss", verbose=1, n_jobs=-1, random_state=2023, refit=True)

# Entraînement
clf_rf_acp_rnd.fit(X_prime_all, y_train)

In [ ]:
clf_rf_acp_final = clf_rf_acp_rnd.best_estimator_
clf_rf_acp_final

In [ ]:
print(f'Meilleurs paramètres: {clf_rf_acp_rnd.best_params_}')
print(f'Meilleur score (mean log loss CV): {-clf_rf_acp_rnd.best_score_}')

In [ ]:
y_train_pred_proba_rf_acp = clf_rf_acp_final.predict_proba(X_prime_all)[:,1]
print(f'AUC = {roc_auc_score(y_train, y_train_pred_proba_rf_acp)}')

| Modèle | Log loss | AUC (refit)
| ------ | ------ | ------
| Régression logisitique | 0.3345   | 0.8174  
| KNN | 0.3532 | 0.8166
| Arbres | 0.3780 | 0.7573
| Bagging (arbres) | 0.3452 | 0.8423
| Gradient boosting | 0.3452 | 0.8221
| Forêts aléatoires | 0.3451 | 0.9796
| Forêts aléatoire + ACP | 0.3067 | 0.9999

In [ ]:
results['Random Forests + ACP'] = y_train_pred_proba_rf_acp
plot_roc_curve(results, y_train)

## C - Sélection du modèle et performances sur le jeu de test

| Modèle | Log loss | AUC (refit)
| ------ | ------ | ------
| Régression logisitique | 0.3345   | 0.8174  
| KNN | 0.3532 | 0.8166
| Arbres | 0.3780 | 0.7573
| Bagging (arbres) | 0.3452 | 0.8423
| Gradient boosting | 0.3452 | 0.8221
| Forêts aléatoires | 0.3451 | 0.9796
| <span style="color: red">Forêts aléatoire + ACP</span> | <span style="color: red">0.3067</span> | <span style="color: red">0.9999</span>

In [ ]:
X_prime_test = np.c_[
    scaler.transform(X_test),
    pca.transform(X_prime_ind)[test_index,:]
]

In [ ]:
X_prime_test.shape

#### Aire sous la courbe

In [ ]:
y_test_pred_proba_best = clf_rf_acp_final.predict_proba(X_prime_test)[:,1]
print(f'AUC = {roc_auc_score(y_test, y_test_pred_proba_best)}')

#### Courbe ROC

In [ ]:
results_test = {}
results_test['BEST'] = y_test_pred_proba_best
plot_roc_curve(results_test, y_test)

## Fin